# Development ideas
This notebok is used for conveniently testing some development ideas and then implementing them in `PhreeqcMatlab`. It does not follow any specific logic and therefore is a fun place to start learning about the internals of `PhreeqcMatlab`.

## Installation
To play with this notebook, follow the following steps:  
First, install [miniconda](https://docs.conda.io/en/latest/miniconda.html) and create a new virtual environment with python 3.8 or 3.9. Note that Matlab 2022a that I use does not support the higher versions of Python but it will likely change in the future in the updated versions of Matlab. The following is a summary of the next steps, i.e. adding conda-forge, installing [Python engine for Matlab](https://se.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html), and installing `matlab kernel` for Jupyter notebook:  
```bash
conda config --add channels conda-forge
conda create -n mymatlab python=3.8 notebook
conda activate mymatlab
cd MATLAB/R2022a/extern/engines/python
python setup.py install
conda install matlab_kernel
jupyter notebook
```

In [60]:
% startup PhreeqcMatlab toolbox
run('../../startup')

PhreeqcMatlab is starting. Checking for the PhreeqcRM library ...
libphreeqcrm.dll library exists.
PhreeqcMatlab started successfully
libphreeqcrm.lib library exists.
PhreeqcMatlab started successfully
libiphreeqc.dll library exists.
IPhreeqc functions are available
libiphreeqc.lib library exists.
IPhreeqc functions are available



## Surfaces
This part deals with the equilibration of a solution with a surface (and later a phase) in `PhreeqcRM` and automatic creation of `SELECTED_OUTPUT` blocks to extract the most relevant properties of the surface.  
First, I create a `PhreeqcRM` instance to play with the functions that read the surface species and types. Then I create some selected output blocks with them and make sure that everything is in the right place. Finally, I will create a new `SurfaceResult` class to store the output of a surface-solution equilibration.

In [61]:
calcite


calcite = 

  Surface with properties:

                         name: "Calcite_CD_MUSIC"
                       number: 1
                         mass: 10
                          scm: "cd_music"
                 site_density: [4.9500 4.9500]
       surface_master_species: [2x1 string]
    surface_species_reactions: [10x1 string]
                        log_k: [10x1 double]
                           dh: [10x1 double]
        specific_surface_area: 2
                  sites_units: "density"
                    edl_model: "Donan"
                edl_thickness: []
            only_counter_ions: 0
                 capacitances: [1.3000 4.5000]
              cd_music_coeffs: [10x5 double]




In [62]:
% let's see if things work fine:
% create a seawater solution
sw = Solution.seawater;
% sw.run;
% or try sw.run_in_phreeqc; both must work and create 
% a report on seawater composition
% create a calcite surface
calcite = Surface.calcite_surface_cd_music;
calcite.mass = 10.0;

calcite.edl_thickness = [];
% Combine the surfaces as a string and run in PhreeqcRM
iph_string = calcite.combine_surface_solution_string(sw)


iph_string =

    'SURFACE_MASTER_SPECIES 
      Chalk_a Chalk_aOH-0.667 
      Chalk_c Chalk_cH+0.667 
      SURFACE_SPECIES 
      Chalk_cH+0.667 = Chalk_cH+0.667 
      log_k 0 
      delta_h 0 
      -cd_music 0  0  0  0  0 
      Chalk_aOH-0.667 = Chalk_aOH-0.667 
      log_k 0 
      delta_h 0 
      -cd_music 0  0  0  0  0 
      Chalk_cH+0.667 = Chalk_c-0.333 + H+ 
      log_k -3.1446 
      delta_h 0 
      -cd_music -1  0  0  0  0 
      Chalk_cH+0.667 + Ca+2 = Chalk_cCa+1.667 + H+ 
      log_k -2.1934 
      delta_h 0 
      -cd_music -1  2  0  0  0 
      Chalk_cH+0.667 + Mg+2 = Chalk_cMg+1.667 + H+ 
      log_k -2.3467 
      delta_h 0 
      -cd_music -1  2  0  0  0 
      Chalk_aOH-0.667 + H+ = Chalk_aOH2+0.333 
      log_k 13.5401 
      delta_h 0 
      -cd_music 1  0  0  0  0 
      Chalk_aOH-0.667 + CO3-2 + H+ = Chalk_aHCO3-0.667 + OH- 
      log_k 9.2433 
      delta_h 0 
      -cd_music 0.6        -0.6           0           0           0 
      Chalk_aOH-0.667 + C

## Running in PhreeqcRM
The following script creates a `PhreeqcRM` instance and initializes it. Try to run the following cell only once to avoid creating many instances of PhreeqcRM without cleaning them up from the memory.

In [63]:
phreeqc_rm = PhreeqcRM(1, 1); % one cell, one thread
phreeqc_rm = phreeqc_rm.RM_Create();

In [64]:
data_file = 'phreeqc.dat';
phreeqc_rm.RM_LoadDatabase(database_file(data_file));
phreeqc_rm.RM_SetSelectedOutputOn(true);
phreeqc_rm.RM_RunString(true, true, true, iph_string);
phreeqc_rm.RM_FindComponents() % always run it first


ans =

    12




## Creating the selected output block
The following Basic functions of Phreeqc are used in the PUNCH block of the SELECTED_OUTPUT data block:
### MOL("HCO3-")
Molality of an aqueous, exchange, or surface species.
### LM("HCO3-")
Log10 of molality of an aqueous, exchange, or surface species.
### LK_SPECIES("HCO3-")
Log10 of the equilibrium constant for an aqueous, exchange, or surface species.
### LA("HCO3-")
Log10 of activity of an aqueous, exchange, or surface species.
### EQUIV_FRAC("(Hfo_w)2Al+", eq, x\\$)
Equivalent fraction of an exchange or surface species relative to the total number of equivalents of exchange or surface sites. The second argument returns the number of sites per mole of species. The third argument returns the site name (Hfo_w in the example). If an exchange or surface species is not found with the given name, the function returns zero; the second argument is zero, and the third argument is an empty string.
### ACT("HCO3-")
Activity of an aqueous, exchange, or surface species.
### EDL("As", "Hfo")
Moles of element in the diffuse layer of a surface. The number of moles does not include the specifically sorbed species. The surface name should be used, not a surface site name (that is, no underscore). The first argument can have several special values, which return information for the surface: “charge”, surface charge, in equivalents; “sigma”, surface charge density, coulombs per square meter; “psi”, potential, Volts; “water”, mass of water in the diffuse layer, kg.

For CD-MUSIC surfaces, charge, sigma and psi can be requested for the 0, 1 and 2 planes:
EDL("Charge", "Goe") # Charge (eq) at the zero-plane of Goe (Goethite)
EDL("Charge1", "Goe") # Charge (eq) at plane 1 of Goe
EDL("Charge2", "Goe") # Charge (eq) at plane 2 of Goe
and similar for “sigma” and “psi”.
### EDL_SPECIES
EDL_SPECIES(surf\\$, count, name\\$, moles, area, thickness)  
Returns the total number of moles of species in the diffuse layer. The The arguments to the function are as follows: surf\\$ is the name of a surface, such as "Hfo", excluding the site type (such as "_s"); count is the number of species in the diffuse layer; name\\$ is an array of size count that contains the names of aqueous species in the diffuse layer of surface surf\\$; moles is an array of size count that contains the number of moles of each aqueous species in the diffuse layer of surface surf\\$; area is the area of the surface in m^2; thickness is the thickness of the diffuse layer in m. The function applies when -donnan or -diffuse_layer is defined in SURFACE calculations.
### SURF("element", "surface")
Number of moles of the element sorbed on the surface. The second argument should be the surface name, not the surface-site name (that is, no underscore). A redox state may be specified; for example, “As” or “As(5)” is permitted.

### SYS("element")

With a single argument, SYS calculates the number of moles of the element in all phases (solution, equilibrium phases, surfaces, exchangers, solid solutions, and gas phase) in the reaction calculation.

### SYS("element", count , name\\$ , type\\$ , moles )

With five arguments, SYS returns the number of moles of the element in all phases in the reaction calculation (solution, equilibrium phases, surfaces, exchangers, solid solutions, and gas phase), and, in addition, returns values for count_species , name\\$ , type\\$ , moles. Count is the dimension of the name\\$ , type\\$ , and moles arrays. Name\\$ is a character array with the name of each species that contains the element. Type\\$ , is a character array with the type of the phase of each species: “aq”, “equi”, “surf”, “ex”, “s_s”, “gas”, or “diff”; where aq is aqueous, equi is equilibrium phase, surf is surface, ex is exchange, s_s is solid solution, gas is gas phase, and diff is surface diffuse layer. Moles is the number of moles of the element in the species (stoichiometry of element times moles of species). The sum of all items in the moles array is equal to the return value of the SYS function.  
The five-argument form of SYS accepts the following arguments in place of “element”:  
“ elements ” returns the total number of moles of elements solution, exchangers, and surfaces in the calculation, other than H and O. Count is number of elements, valence states, exchangers, and surfaces. Name\\$ contains the element name. Type\\$ contains the type for each array item: “dis” for dissolved, “ex” for exchange, and “surf” for surface. Moles contains the number of moles of the element in each type of phase (stoichiometry of element times moles of species).  
“ phases ” returns the maximum saturation index of all pure phases appropriate for the calculation. Count is number of pure phases. Name\\$ contains the phase names as defined in the PHASES data block. Type\\$ is “phase”. Moles contains the saturation index for the phases.  
“ aq ” returns the sum of moles of all aqueous species in the calculation. Count is number of aqueous species. Name\\$ contains the aqueous species names. Type\\$ is “aq”. Moles contains the moles of species.  
“equi” returns the sum of moles of all equilibrium phases in the calculation. Count is number of equilibrium phases. Name\\$ contains the equilibrium phase names. Type\\$ is “equi”. Moles contains the moles of each equilibrium phase.  
“ ex ” returns the sum of moles of all exchange species in the calculation. Count is number of exchange species. Name\\$ contains the exchange species names. Type\\$ is “ex”. Moles contains the moles of species.  
“kin” returns the sum of moles of all kinetic reactants in the calculation. Count is number of kinetic reactants. Name\\$ contains the kinetic reactant names. Type\\$ is “kin”. Moles contains the moles of each kinetic reactant. The chemical formula used in the kinetic reaction can be determined by using a reaction name from Name\\$ as the first argument of the KINETICS_FORMULA\\$ Basic function.  
“ surf ” returns the sum of moles of all surface species in the calculation. Count is number of surface species. Name\\$ contains the surface species names. Type\\$ is “surf”. Moles contains the moles of species.  
“ s_s ” returns sum of moles of all solid-solution components in the calculation. Count is number of solid-solution components. Name\\$ contains the names of the solid-solution components. Type\\$ is “s_s”. Moles contains the moles of components.  
“ gas ” returns sum of moles of all gas components in the calculation. Count is number of gas components. Name\\$ contains names of the gas components. Type\\$ is “gas”. Moles contains the moles of gas components

In [65]:
element_names = phreeqc_rm.GetComponents()
surf_sp_name = phreeqc_rm.GetSurfaceSpeciesNames()
surf_type = phreeqc_rm.GetSurfaceTypes
b = split(surf_type{1}, '_');
surf_name = strjoin(b(1:end-1), '_')
sur_sp_list = strjoin(phreeqc_rm.GetSurfaceSpeciesNames())


element_names =

  12x1 cell array

    {'H2O'   }
    {'H'     }
    {'O'     }
    {'Charge'}
    {'C'     }
    {'Ca'    }
    {'Cl'    }
    {'K'     }
    {'Mg'    }
    {'Na'    }
    {'S'     }
    {'Si'    }


surf_sp_name =

  10x1 cell array

    {'Chalk_aCO3-1.667' }
    {'Chalk_aHCO3-0.667'}
    {'Chalk_aOH-0.667'  }
    {'Chalk_aOH2+0.333' }
    {'Chalk_aSO4-1.667' }
    {'Chalk_c-0.333'    }
    {'Chalk_cCa+1.667'  }
    {'Chalk_cH+0.667'   }
    {'Chalk_cMg+1.667'  }
    {'Chalk_cNa+0.667'  }


surf_type =

  10x1 cell array

    {'Chalk_a'}
    {'Chalk_a'}
    {'Chalk_a'}
    {'Chalk_a'}
    {'Chalk_a'}
    {'Chalk_c'}
    {'Chalk_c'}
    {'Chalk_c'}
    {'Chalk_c'}
    {'Chalk_c'}


surf_name =

    'Chalk'


sur_sp_list =

    'Chalk_aCO3-1.667 Chalk_aHCO3-0.667 Chalk_aOH-0.667 Chalk_aOH2+0.333 Chalk_aSO4-1.667 Chalk_c-0.333 Chalk_cCa+1.667 Chalk_cH+0.667 Chalk_cMg+1.667 Chalk_cNa+0.667'




In [66]:
a = 'chalk_fast_a';
b = split(a, '_');
strjoin(b(1:end-1), '_')
sur_sp_list = strjoin(phreeqc_rm.GetSurfaceSpeciesNames())
element_names(4:end)


ans =

    'chalk_fast'


sur_sp_list =

    'Chalk_aCO3-1.667 Chalk_aHCO3-0.667 Chalk_aOH-0.667 Chalk_aOH2+0.333 Chalk_aSO4-1.667 Chalk_c-0.333 Chalk_cCa+1.667 Chalk_cH+0.667 Chalk_cMg+1.667 Chalk_cNa+0.667'


ans =

  9x1 cell array

    {'Charge'}
    {'C'     }
    {'Ca'    }
    {'Cl'    }
    {'K'     }
    {'Mg'    }
    {'Na'    }
    {'S'     }
    {'Si'    }




In [67]:
% sw.solution_selected_output
ind_charge = find(strcmpi(element_names, 'Charge'));
surf_call = strjoin(cellfun(@(x)(['SURF("' x '","' surf_name '")']), element_names(ind_charge+1:end), 'UniformOutput', false))
edl_special = [{'Charge'}; {'Charge1'}; {'Charge2'}; {'sigma'}; {'sigma1'}; {'sigma2'}; {'psi'}; {'psi1'}; {'psi2'}; {'water'}]
edl_in = [element_names(ind_charge+1:end); edl_special]
edl_call = strjoin(cellfun(@(x)(['EDL("' x '","' surf_name '")']), edl_in, 'UniformOutput', false))


surf_call =

    'SURF("C","Chalk") SURF("Ca","Chalk") SURF("Cl","Chalk") SURF("K","Chalk") SURF("Mg","Chalk") SURF("Na","Chalk") SURF("S","Chalk") SURF("Si","Chalk")'


edl_special =

  10x1 cell array

    {'Charge' }
    {'Charge1'}
    {'Charge2'}
    {'sigma'  }
    {'sigma1' }
    {'sigma2' }
    {'psi'    }
    {'psi1'   }
    {'psi2'   }
    {'water'  }


edl_in =

  18x1 cell array

    {'C'      }
    {'Ca'     }
    {'Cl'     }
    {'K'      }
    {'Mg'     }
    {'Na'     }
    {'S'      }
    {'Si'     }
    {'Charge' }
    {'Charge1'}
    {'Charge2'}
    {'sigma'  }
    {'sigma1' }
    {'sigma2' }
    {'psi'    }
    {'psi1'   }
    {'psi2'   }
    {'water'  }


edl_call =

    'EDL("C","Chalk") EDL("Ca","Chalk") EDL("Cl","Chalk") EDL("K","Chalk") EDL("Mg","Chalk") EDL("Na","Chalk") EDL("S","Chalk") EDL("Si","Chalk") EDL("Charge","Chalk") EDL("Charge1","Chalk") EDL("Charge2","Chalk") EDL("sigma","Chalk") EDL("sigma1","Chalk") EDL("sigma2","Chalk") EDL("psi","Chalk") EDL(

As you can see from the above results, the RM_FindComponents must be called before retrieving any other species-related information from the PhreeqcRM instance. Note that there is no need to call RM_RunCells function and only running the phreeqc string or file is enough for acquiring the data required for creating the selected output block. It might even be possible to retrieve the selected output values, that I will test in the following cells.

In [68]:
so_string = strjoin(["\nSELECTED_OUTPUT" num2str(calcite.number) "\n"]);
so_string = strjoin([so_string  "-reset false\n"]);
so_string = strjoin([so_string "-molalities" sur_sp_list "\n"]);
so_string = strjoin([so_string  "USER_PUNCH\n"]);
so_string = strjoin([so_string  "-headings "  element_names(ind_charge+1:end)' edl_in' "\n"]);
so_string = strjoin([so_string  "10 PUNCH" surf_call edl_call "\n"]);
so_string = strjoin([so_string  "END"]);
so_string = sprintf(char(so_string));
new_string = [iph_string so_string]


new_string =

    'SURFACE_MASTER_SPECIES 
      Chalk_a Chalk_aOH-0.667 
      Chalk_c Chalk_cH+0.667 
      SURFACE_SPECIES 
      Chalk_cH+0.667 = Chalk_cH+0.667 
      log_k 0 
      delta_h 0 
      -cd_music 0  0  0  0  0 
      Chalk_aOH-0.667 = Chalk_aOH-0.667 
      log_k 0 
      delta_h 0 
      -cd_music 0  0  0  0  0 
      Chalk_cH+0.667 = Chalk_c-0.333 + H+ 
      log_k -3.1446 
      delta_h 0 
      -cd_music -1  0  0  0  0 
      Chalk_cH+0.667 + Ca+2 = Chalk_cCa+1.667 + H+ 
      log_k -2.1934 
      delta_h 0 
      -cd_music -1  2  0  0  0 
      Chalk_cH+0.667 + Mg+2 = Chalk_cMg+1.667 + H+ 
      log_k -2.3467 
      delta_h 0 
      -cd_music -1  2  0  0  0 
      Chalk_aOH-0.667 + H+ = Chalk_aOH2+0.333 
      log_k 13.5401 
      delta_h 0 
      -cd_music 1  0  0  0  0 
      Chalk_aOH-0.667 + CO3-2 + H+ = Chalk_aHCO3-0.667 + OH- 
      log_k 9.2433 
      delta_h 0 
      -cd_music 0.6        -0.6           0           0           0 
      Chalk_aOH-0.667 + C

In [69]:
phreeqc_rm.RM_RunString(true, true, true, new_string);
phreeqc_rm.RM_FindComponents(); % always run it first
phreeqc_rm.RM_SetSelectedOutputOn(true);
phreeqc_rm.RM_SetComponentH2O(true);
phreeqc_rm.RM_SetUnitsSolution(2);
phreeqc_rm.RM_SetSpeciesSaveOn(false);
ic1 = -1*ones(7, 1);
ic2 = -1*ones(7, 1);
% 1 solution, 2 eq phase, 3 exchange, 4 surface, 5 gas, 6 solid solution, 7 kinetic
f1 = ones(7, 1);
ic1(1) = sw.number;              % Solution seawater
ic1(4) = calcite.number;         % Surface calcite
phreeqc_rm.RM_InitialPhreeqc2Module(ic1, ic2, f1);
phreeqc_rm.RM_RunCells();
t_out = phreeqc_rm.GetSelectedOutputTable(calcite.number);
phreeqc_rm.RM_Destroy()


ans =

    'IRM_OK'




## Running the cell with selected output

In [70]:
t_out.keys
t_out.values


ans =

  1x28 cell array

  Columns 1 through 12

    {'C'}    {'Ca'}    {'Charge'}    {'Charge1'}    {'Charge2'}    {'Cl'}    {'K'}    {'Mg'}    {'Na'}    {'S'}    {'Si'}    {'m_Chalk_aCO3-1...'}

  Columns 13 through 18

    {'m_Chalk_aHCO3-...'}    {'m_Chalk_aOH-0....'}    {'m_Chalk_aOH2+0...'}    {'m_Chalk_aSO4-1...'}    {'m_Chalk_c-0.33...'}    {'m_Chalk_cCa+1....'}

  Columns 19 through 28

    {'m_Chalk_cH+0.6...'}    {'m_Chalk_cMg+1....'}    {'m_Chalk_cNa+0....'}    {'psi'}    {'psi1'}    {'psi2'}    {'sigma'}    {'sigma1'}    {'sigma2'}    {'water'}


ans =

  1x28 cell array

  Columns 1 through 9

    {[4.4179e-07]}    {[2.0559e-06]}    {[1.4079e-06]}    {[3.8586e-06]}    {[0]}    {[1.1545e-04]}    {[2.0748e-06]}    {[1.0637e-05]}    {[9.5201e-05]}

  Columns 10 through 18

    {[5.9970e-06]}    {[1.4778e-08]}    {[1.8775e-06]}    {[9.1356e-07]}    {[2.0646e-08]}    {[0.0016]}    {[3.8478e-05]}    {[0.0016]}    {[7.1688e-06]}

  Columns 19 through 28

    {[4.9529e-09]}    